# Data analysis
In below notebook, we analyze data and write some conclusions.

In [0]:
sas_token = "sp=rcwdl&st=2026-01-02T21:18:29Z&se=2026-01-03T05:33:29Z&spr=https&sv=2024-11-04&sr=c&sig=U85UUMwZI2b3er9g1fw3DRdJu41ma0XSWv911SXkXh0%3D"
storage_account_name = "newadbprojektkakastorage"
container_name = "data"

spark.conf.set(
    f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "SAS"
)
spark.conf.set(
    f"fs.azure.sas.token.provider.type.{storage_account_name}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider"
)
spark.conf.set(
    f"fs.azure.sas.fixed.token.{storage_account_name}.dfs.core.windows.net",
    sas_token
)

base_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net"
print(f"Skonfigurowano dostęp do: {base_path}")

Skonfigurowano dostęp do: abfss://data@newadbprojektkakastorage.dfs.core.windows.net


## Analiza zbioru złączonego z df_flights i df_weather

In [0]:
file_path = "dbfs:/FileStore/tables/df_weather_jfk.csv"

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

print("Wczytano dane")
display(df.limit(5))

df.printSchema()

Wczytano dane


DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,DEPARTURE_DELAY,DISTANCE,CANCELLED,ORIGIN_AIRPORT,WEATHER_DELAY,ScheduledTimestamp,DeparturedTimestamp,HOURLYPrecip,HOURLYVISIBILITY,HOURLYWindSpeed
1,DL,2338,-3,1069,0,JFK,null,2015-01-05T16:00:00Z,2015-01-05T15:57:00Z,0.0,10.0,15.0
1,AA,67,9,1598,0,JFK,null,2015-01-05T16:05:00Z,2015-01-05T16:14:00Z,0.0,10.0,15.0
1,AA,65,5,1391,0,JFK,null,2015-01-05T16:15:00Z,2015-01-05T16:20:00Z,0.0,10.0,15.0
1,VX,27,2,2586,0,JFK,null,2015-01-05T16:30:00Z,2015-01-05T16:32:00Z,0.0,10.0,15.0
1,AA,133,0,2475,0,JFK,null,2015-01-05T16:30:00Z,2015-01-05T16:30:00Z,0.0,10.0,15.0


root
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- WEATHER_DELAY: integer (nullable = true)
 |-- ScheduledTimestamp: timestamp (nullable = true)
 |-- DeparturedTimestamp: timestamp (nullable = true)
 |-- HOURLYPrecip: double (nullable = true)
 |-- HOURLYVISIBILITY: double (nullable = true)
 |-- HOURLYWindSpeed: double (nullable = true)



In [0]:
from pyspark.sql.functions import min, max, col

df_bad_weather = df.filter(col("WEATHER_DELAY") > 0)
display(df_bad_weather)

DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,DEPARTURE_DELAY,DISTANCE,CANCELLED,ORIGIN_AIRPORT,WEATHER_DELAY,ScheduledTimestamp,DeparturedTimestamp,HOURLYPrecip,HOURLYVISIBILITY,HOURLYWindSpeed
1,VX,251,0,2248,0,JFK,18,2015-01-05T09:45:00Z,2015-01-05T09:45:00Z,0.0,10.0,21.0
2,VX,411,12,2475,0,JFK,40,2015-01-06T13:00:00Z,2015-01-06T13:12:00Z,0.02,2.5,9.0
7,B6,915,17,2586,0,JFK,16,2015-02-15T20:30:00Z,2015-02-15T20:47:00Z,0.0,10.0,24.0
7,VX,501,-6,1069,0,JFK,27,2015-01-18T08:00:00Z,2015-01-18T07:54:00Z,0.01,10.0,6.0
1,VX,415,-3,2475,0,JFK,27,2015-03-02T20:15:00Z,2015-03-02T20:12:00Z,0.0,10.0,15.0
4,VX,413,-3,2475,0,JFK,16,2015-02-19T16:55:00Z,2015-02-19T16:52:00Z,0.0,10.0,28.0
1,VX,407,9,2475,0,JFK,27,2015-02-09T09:20:00Z,2015-02-09T09:29:00Z,0.02,10.0,17.0
2,VX,251,-2,2248,0,JFK,19,2015-03-03T09:45:00Z,2015-03-03T09:43:00Z,0.0,10.0,0.0
2,DL,480,16,944,0,JFK,16,2015-02-17T12:59:00Z,2015-02-17T13:15:00Z,0.01,9.0,10.0
7,VX,413,-1,2475,0,JFK,16,2015-02-22T16:55:00Z,2015-02-22T16:54:00Z,0.0,8.0,11.0


Powyżej widać, że żaden lot nie został odwołany, gdy WEATHER_DELAY > 0. Możemy stwierdzić zatem, że WEATHER_DELAY dotyczy wyłącznie opóźnień.

In [0]:
df.select(
    min(col("ScheduledTimestamp")).alias("Najstarsza_Data"),
    max(col("ScheduledTimestamp")).alias("Najmlodsza_Data")
).display()

Najstarsza_Data,Najmlodsza_Data
2015-01-01T05:45:00Z,2015-12-31T23:59:00Z


In [0]:
print(f"Liczba wszystkich lotów: {df.count()}")
print(f"Liczba opóźnionych lotów: {df.filter(col("DEPARTURE_DELAY")>0).count()}")
print(f"Liczba lotów, które wyleciały wcześniej niż powinny: {df.filter(col("DEPARTURE_DELAY")<0).count()}")

Liczba wszystkich lotów: 64777
Liczba opóźnionych lotów: 14806
Liczba lotów, które wyleciały wcześniej niż powinny: 44874


Loty, które wyleciały wcześniej zaburzają nam analizę wpływu pogody na opóźnienia.

In [0]:
df = df.filter(col("DEPARTURE_DELAY")>=0)
display(df.summary())

summary,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,DEPARTURE_DELAY,DISTANCE,CANCELLED,ORIGIN_AIRPORT,WEATHER_DELAY,HOURLYPrecip,HOURLYVISIBILITY,HOURLYWindSpeed
count,19903,19903,19903,19903,19903,19903,19903,30,19903,19903,19903
mean,4.038285685575039,null,898.9001658041501,4.9570416520122595,1532.6167914384766,3.5170577299904537E-4,null,24.733333333333334,0.0027186856252826197,9.400147716424662,11.428880068331408
stddev,2.006807156329876,null,850.5517816479158,4.992559384900814,840.4754398803761,0.018750993086069746,null,12.122743894854409,0.021365306692347978,1.9608284536353995,5.79912408936341
min,1,AA,1,0,173,0,JFK,15,0.0,0.0,0.0
25%,2,null,359,0,944,0,null,17,0.0,10.0,8.0
50%,4,null,491,3,1576,0,null,19,0.0,10.0,11.0
75%,6,null,1295,8,2454,0,null,28,0.0,10.0,15.0
max,7,VX,3684,17,4983,1,JFK,70,0.61,10.0,37.0


## Czy w któryś miesiąc opóźnienia są większe (wakacje, Boże Narodzenie? śnieg, mgła?)?


In [0]:
from pyspark.sql.functions import month, col, avg

df_grouped = df.withColumn("Month", month(col("ScheduledTimestamp"))) \
               .groupBy("Month") \
               .agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")) \
               .orderBy("Month")

display(df_grouped)

Month,Avg_Delay
1,4.934189406099518
2,5.37359900373599
3,5.264961915125136
4,5.12219598583235
5,4.974056603773585
6,4.8
7,4.687377690802348
8,4.929475587703436
9,4.721236028928336
11,4.5988130563798215


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import month, col, avg

df_grouped = df_bad_weather.withColumn("Month", month(col("ScheduledTimestamp"))) \
               .groupBy("Month") \
               .agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")) \
               .orderBy("Month")

display(df_grouped)

Month,Avg_Delay
1,3.0
2,6.076923076923077
3,-0.5333333333333333
4,-2.3
5,-2.375
6,-7.0
7,8.0
8,-1.0
9,3.6666666666666665
11,-2.0


Databricks visualization. Run in Databricks to view.

## Czy w weekend opóźnienia są większe?


In [0]:
from pyspark.sql.functions import when, col
df_features = df.withColumn("IsWeekend", when(col("DAY_OF_WEEK").isin([6, 7]), 1).otherwise(0))


display(df_features.groupBy("IsWeekend").agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")))

df_weekly_analysis = df_features.groupBy("DAY_OF_WEEK") \
    .agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")) \
    .orderBy("DAY_OF_WEEK")

display(df_weekly_analysis)

IsWeekend,Avg_Delay
1,4.973496976165066
0,4.950563686016386


Databricks visualization. Run in Databricks to view.

DAY_OF_WEEK,Avg_Delay
1,5.051528684300927
2,5.0294916569654635
3,4.899052478134111
4,4.874201143625967
5,4.908647594278284
6,5.0439387995292275
7,4.915066710055321


Databricks visualization. Run in Databricks to view.

## Czy loty długodystansowe są narażone na większe opóźnienia przy różnych warunkach atmosferycznych?


In [0]:
from pyspark.sql.functions import col, avg, floor

df_distance_bins = df.withColumn("Distance_Bin", floor(col("DISTANCE") / 100) * 100)

df_distance = df_distance_bins.groupBy("Distance_Bin") \
                         .agg(avg("DEPARTURE_DELAY").alias("Avg_Delay")) \
                         .orderBy("Distance_Bin")

display(df_distance)

Distance_Bin,Avg_Delay
100,5.119371727748691
200,5.408388520971302
300,4.77979274611399
400,5.043227665706052
500,5.230650154798762
600,4.9216417910447765
700,5.1711131554737815
800,4.963917525773196
900,4.8513274336283185
1000,5.217137592137592


Databricks visualization. Run in Databricks to view.

## Czy któraś linia lotnicza ma większe opóźnienia niż pozostałe w złe warunki pogodowe?


In [0]:
from pyspark.sql.functions import col, avg, count, desc

airline_stats = df_bad_weather.groupBy("AIRLINE") \
    .agg(
        avg("WEATHER_DELAY").alias("Avg_Weather_Delay"),  # Średnie opóźnienie przez pogodę
        avg("DEPARTURE_DELAY").alias("Avg_Total_Delay"),  # Średnie całkowite opóźnienie (gdy była zła pogoda)
        count("*").alias("Flight_Count")                  # Liczba takich lotów 
    ) \
    .orderBy(col("Avg_Total_Delay").desc()) # Sortujemy od najgorszej

display(airline_stats)

airline_stats = df.groupBy("AIRLINE") \
    .agg(
        avg("DEPARTURE_DELAY").alias("Avg_Total_Delay"),  # Średnie całkowite opóźnienie (gdy była zła pogoda)
        count("*").alias("Flight_Count")                  # Liczba takich lotów 
    ) \
    .orderBy(col("Avg_Total_Delay").desc()) # Sortujemy od najgorszej

display(airline_stats)

AIRLINE,Avg_Weather_Delay,Avg_Total_Delay,Flight_Count
B6,16.0,17.0,1
DL,16.5,16.75,4
AA,16.0,16.5,2
VX,26.02857142857143,-1.1571428571428573,70
HA,23.0,-5.0,1


AIRLINE,Avg_Total_Delay,Flight_Count
MQ,5.468682505399568,463
US,5.3575949367088604,316
DL,5.261417580618759,6109
B6,4.945270741343249,8606
AA,4.568205484206873,2881
VX,4.401388888888889,720
UA,4.277266754270697,761
AS,3.210526315789474,19
HA,3.142857142857143,28


## Wpływ prędkości wiatru na opóźnienia i odwołania lotów

In [0]:
from pyspark.sql.functions import round, avg, col, sum, count

df_wind_analysis = df.filter(col("DEPARTURE_DELAY").isNotNull()) \
    .groupBy(round(col("HOURLYWindSpeed")).alias("WindSpeed")) \
    .agg(avg("DEPARTURE_DELAY").alias("AvgDelay"), count("*").alias("FlightCount")) \
    .orderBy("WindSpeed")

display(df_wind_analysis)

df_wind_analysis_cancel = df \
    .groupBy(round(col("HOURLYWindSpeed")).alias("WindSpeed")) \
    .agg(
        avg("CANCELLED").alias("CancellationRate"),  # Zamiana 0/1 na procent (0.12 = 12%)
        sum("CANCELLED").alias("TotalCancelled"),    # Ile fizycznie odwołano
        count("*").alias("TotalFlights")             # Ile było wszystkich lotów przy tym wietrze
    ) \
    .filter(col("TotalFlights") > 10) \
    .orderBy("WindSpeed")

display(df_wind_analysis_cancel)

WindSpeed,AvgDelay,FlightCount
0.0,4.52322738386308,818
3.0,4.557911908646004,613
5.0,4.54312114989733,974
6.0,4.926333615580017,1181
7.0,4.909785932721713,1308
8.0,4.9507481296758105,1604
9.0,4.904166666666667,1680
10.0,5.022498519834222,1689
11.0,5.054957507082153,1765
13.0,5.179375,1600


Databricks visualization. Run in Databricks to view.

WindSpeed,CancellationRate,TotalCancelled,TotalFlights
0.0,0.0,0,818
3.0,0.0,0,613
5.0,0.0,0,974
6.0,8.46740050804403E-4,1,1181
7.0,7.645259938837921E-4,1,1308
8.0,0.0,0,1604
9.0,0.0,0,1680
10.0,0.0,0,1689
11.0,0.0,0,1765
13.0,0.0,0,1600


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import round, avg, col

df_wind_analysis = df_bad_weather.filter(col("DEPARTURE_DELAY").isNotNull()) \
    .groupBy(round(col("HOURLYWindSpeed")).alias("WindSpeed")) \
    .agg(avg("DEPARTURE_DELAY").alias("AvgDelay"), count("*").alias("FlightCount")) \
    .orderBy("WindSpeed")

display(df_wind_analysis)


df_wind_analysis = df_bad_weather \
    .groupBy(round(col("HOURLYWindSpeed")).alias("WindSpeed")) \
    .agg(
        avg("CANCELLED").alias("CancellationRate"),  # Zamiana 0/1 na procent (0.12 = 12%)
        sum("CANCELLED").alias("TotalCancelled"),    # Ile fizycznie odwołano
        count("*").alias("TotalFlights")             # Ile było wszystkich lotów przy tym wietrze
    ) \
    .filter(col("TotalFlights") > 10) \
    .orderBy("WindSpeed")

display(df_wind_analysis_cancel)

WindSpeed,AvgDelay,FlightCount
0.0,-1.1666666666666667,6
3.0,8.0,2
5.0,-2.0,2
6.0,-2.2,5
7.0,-3.5,2
8.0,-2.8333333333333335,6
9.0,5.0,5
10.0,2.5,4
11.0,-0.5,4
13.0,3.0,5


Databricks visualization. Run in Databricks to view.

WindSpeed,CancellationRate,TotalCancelled,TotalFlights
0.0,0.0,0,818
3.0,0.0,0,613
5.0,0.0,0,974
6.0,8.46740050804403E-4,1,1181
7.0,7.645259938837921E-4,1,1308
8.0,0.0,0,1604
9.0,0.0,0,1680
10.0,0.0,0,1689
11.0,0.0,0,1765
13.0,0.0,0,1600


Databricks visualization. Run in Databricks to view.

## Wpływ widoczności na opóźnienia i odwołania lotów

In [0]:
from pyspark.sql.functions import round, avg, col, count

df_vis_analysis = df.filter(col("DEPARTURE_DELAY").isNotNull()) \
    .groupBy(round(col("HOURLYVISIBILITY"), 1).alias("Visibility")) \
    .agg(
        avg("DEPARTURE_DELAY").alias("AvgDelay"), 
        count("*").alias("FlightCount")
    ) \
    .filter(col("FlightCount") > 10) \
    .orderBy("Visibility")

display(df_vis_analysis)

Visibility,AvgDelay,FlightCount
0.0,6.132075471698113,53
0.1,4.463917525773196,97
0.3,4.325,80
0.5,4.0843373493975905,83
0.8,4.6716417910447765,67
1.0,4.64,100
1.3,3.962962962962963,27
1.5,6.229508196721311,61
2.0,4.991935483870968,124
2.5,5.382716049382716,81


Databricks visualization. Run in Databricks to view.

In [0]:
df_vis_analysis_cancel = df \
    .groupBy(round(col("HOURLYVISIBILITY")).alias("Visibility")) \
    .agg(
        avg("CANCELLED").alias("CancellationRate"),  # Zamiana 0/1 na procent (0.12 = 12%)
        sum("CANCELLED").alias("TotalCancelled"),    # Ile fizycznie odwołano
        count("*").alias("TotalFlights")             # Ile było wszystkich lotów przy tym wietrze
    ) \
    .filter(col("TotalFlights") > 5) \
    .orderBy("Visibility")

display(df_vis_analysis_cancel)

Visibility,CancellationRate,TotalCancelled,TotalFlights
0.0,0.0,0,230
1.0,0.0,0,277
2.0,0.0,0,185
3.0,0.005291005291005291,1,189
4.0,0.0,0,68
5.0,0.0,0,222
6.0,0.0,0,229
7.0,0.0,0,267
8.0,0.0,0,322
9.0,0.0,0,373


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import round, avg, col, count

df_vis_analysis = df_bad_weather.filter(col("DEPARTURE_DELAY").isNotNull()) \
    .groupBy(round(col("HOURLYVISIBILITY"), 1).alias("Visibility")) \
    .agg(
        avg("DEPARTURE_DELAY").alias("AvgDelay"), 
        count("*").alias("FlightCount")
    ) \
    .orderBy("Visibility")

display(df_vis_analysis)

Visibility,AvgDelay,FlightCount
0.0,-0.5,2
0.1,-1.0,1
0.5,-1.0,1
0.8,8.5,2
1.0,-4.0,1
2.5,12.0,1
3.0,-3.3333333333333335,3
5.0,-1.6666666666666667,3
6.0,-2.0,3
8.0,-1.5,2


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import col, avg, hour

df_time_analysis = (
    df.filter(col("DEPARTURE_DELAY").isNotNull())
    .withColumn("Hour", hour(col("ScheduledTimestamp")))
    .groupBy("Hour")
    .agg(avg("DEPARTURE_DELAY").alias("AvgDelay"))
    .orderBy("Hour")
)

display(df_time_analysis)

Hour,AvgDelay
5,3.457894736842105
6,3.6226215644820297
7,4.1599115696389095
8,4.234965902045877
9,4.508414526129318
10,4.896866840731071
11,4.462113127001067
12,4.785714285714286
13,5.085923217550274
14,5.0336


Databricks visualization. Run in Databricks to view.